In [ ]:
import PyPDF2

with open('./AIPF Emergenetics Profile/Profile_Narrative_Watson_Chua.pdf', 'rb') as f:
    # creating a page object
    pdfReader = PyPDF2.PdfFileReader(f)
    pageObj = pdfReader.getPage(0)
    # extracting text from page
    text = pageObj.extractText()

In [ ]:
text

In [ ]:
import re
def extract_content_from_profile(profile):
    attributes = ['CONCEPTUAL', 'ANALYTICAL', 'STRUCTURAL', 'SOCIAL']
    results = {}
    for attr in attributes:
        value = re.findall(re.escape(attr) + r' = (\d+)%', text)[0]
        results[attr] = value
        results['NAME'] = text.split('\n')[-4].split('-')[0].strip()
    return results



In [ ]:
from glob import glob
all_profiles = []
filepaths = glob('./AIPF Emergenetics Profile/*.pdf')
for fp in filepaths:
    with open(fp, 'rb') as f:
        pdfReader = PyPDF2.PdfFileReader(f)
        pageObj = pdfReader.getPage(0)
        text = pageObj.extractText()
        if not text.startswith('HOW YOU THINK'):
            print(text)
            continue
        profile = extract_content_from_profile(text)
        all_profiles.append(profile)
all_profiles

In [ ]:
import pandas as pd
df_profiles = pd.DataFrame(all_profiles)

In [ ]:
df_profiles.loc[df_profiles['NAME']=='WATSON CHUA']

In [ ]:
df_profiles.to_csv('profile.csv', index=False)

## Google Auth

In [ ]:
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://www.googleapis.com/auth/drive.readonly']

credentials = ServiceAccountCredentials.from_json_keyfile_name('gdrive-access-368306-1d07a7f84034.json', scope)

# https://developers.google.com/drive/api/v3/quickstart/python
service = build('drive', 'v3', credentials=credentials)

# Call the Drive v3 API
results = service.files().list(
    fields="*",corpora = 'drive',supportsAllDrives = True, driveId = "1uoapecGhphu5AF5Lqd5jYhQYGBFO2j6u", includeItemsFromAllDrives = True).execute()
items = results.get('files', [])

if not items:
    print('No files found.')
else:
    print('Files:')
    for item in items:
        print(u'{0} ({1})'.format(item['name'], item['id']))

In [ ]:
res = service.drives()

In [ ]:
res

In [ ]:
# Textract output
import json
with open('./watson_textract.json', 'r') as f:
    data = json.load(f)

In [ ]:
all_text = []
for block in data['Blocks']:
    # print(block.keys())
    if "Text" in block:
        all_text.append(block["Text"])
        # print(block["Text"])
text_str = '\n'.join(all_text)

In [ ]:
text_str

In [ ]:
import re
attributes = ['CONCEPTUAL', 'ANALYTICAL', 'STRUCTURAL', 'SOCIAL']
# results = {}
for attr in attributes:
    value = re.findall(re.escape(attr) + r' = (\d+)%', text_str)[0]
    print(value)

In [ ]:
percentile_pattern = r"\nAnalytical\n(\d+)\nStructural\n(\d+)\nSocial\n(\d+)\nConceptual\n(\d+)\nExpressiveness\n(\d+)\nQuiet\nIntrospective\nReserved\nTalkative\nGregarious\nAssertiveness\n(\d+)\nPeacekeeping\nEasygoing\nCompetitive\nForceful\nDriving\nFlexibility\n(\d+)"
re.findall(percentile_pattern, text_str)[0]

In [29]:
from tika import parser

In [31]:
parsed_pdf = parser.from_file("./AIPF Emergenetics Profile/Profile_Narrative_Watson_Chua.pdf")


2022-11-15 17:34:24,901 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


In [32]:
text= parsed_pdf['content']

In [33]:
text

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nHOW YOU THINK: PERCENTAGES\nH\n\no\nw\n\n y\no\n\nu\n T\n\nh\nin\n\nk\n: \n\nP\ne\nrc\n\ne\nn\n\nti\nle\n\ns\nH\n\no\nw\n\n y\no\n\nu\n B\n\ne\nh\n\na\nv\ne\n: \n\nP\ne\nrc\n\ne\nn\n\nti\nle\n\ns\n\nGeneral Population   0          10         20          30         40         50          60         70         80          90         100\n\nQuiet              Introspective                Reserved               Talkative             Gregarious\n\nPeacekeeping           Easygoing             Competitive             Forceful               Driving\n\nFocused           Firm           Adaptable          Accommodating          Welcomes Change\n\nStructural\n\nSocial\n\nConceptual\n\nExpressiveness\n\nAssertiveness\n\n 19\n\n 5\n\n 69\n\nCONCEPTUAL = 38%ANALYTICAL = 49%\n\nSTRUCTURAL = 10%\n\nImaginative\n\nIntuitive about ideas\n\nVisionary\n\nEnjoys the unusual\n\nLearns by experimenting\n\nSOCIAL = 3%\n\nHOW YOU COMPA

In [36]:
import re
motto_pattern = r"\nYour Motto: (.*)\n"
re.findall(motto_pattern, text)[0][1:-1]

'I see the forest.'

In [ ]:
import re

# analytical_percentile_pattern = r"You rank in the (\d+) percentile in Analytical thinking"
# structural_percentile_pattern = r"you rank in the (\d+) percentile in Structural thinking"
# social_percentile_pattern = r"You rank in the (\d+) percentile in Social thinking" 
# conceptual_percentile_pattern = r"Conceptual thinking, you rank in the (\d+) percentile"

percentile_patterns = r"The Percentiles\n\nAnalytical (\d+)%ile\n\nStructural (\d+)%ile\n\nSocial (\d+)%ile\n\nConceptual (\d+)%ile\n\nBar charts in four colors show your Thinking Attributes in percentiles."
motto_pattern = r"\nYour Motto: (.*)\n"

conceptual_pattern = r"CONCEPTUAL = (\d+)%"
analytical_pattern = r"ANALYTICAL = (\d+)%"
structural_pattern = r"STRUCTURAL = (\d+)%"
social_pattern = r"SOCIAL = (\d+)%"


other_patterns = r"Expressiveness (\d+)%ile\n\nAssertiveness (\d+)%ile\n\nFlexibility (\d+)%ile"
name_pattern = r"Congratulations,(.*)!"

def extract_values(text):
    # test = re.findall(analytical_percentile_pattern, text) + re.findall(structural_percentile_pattern, text) + re.findall(social_percentile_pattern, text) + re.findall(conceptual_percentile_pattern, text)
    # print(test)
    name = re.findall(name_pattern, text)[0]
    analytical_percentile, structural_percentile, social_percentile, conceptual_percentile = re.findall(percentile_patterns, text)[0]
    analytical_percentage = re.findall(analytical_pattern, text)[0]
    structural_percentage = re.findall(structural_pattern, text)[0]
    social_percentage = re.findall(social_pattern, text)[0]
    conceptual_percentage = re.findall(conceptual_pattern, text)[0]
    expressiveness, assertiveness, flexibility = re.findall(other_patterns, text)[0]
    motto = re.findall(motto_pattern, text)[0][1:-1]

    return {
        'Name': name.title().strip(),
        'Motto': motto.strip(),
        'Analytical': analytical_percentage, 
        'Conceptual': conceptual_percentage,
        'Structural': structural_percentage, 
        'Social': social_percentage, 
        'Analytical Percentile': analytical_percentile,
        'Conceptual Percentile': conceptual_percentile, 
        'Structural Percentile': structural_percentile, 
        'Social Percentile': social_percentile, 
        'Expressiveness': expressiveness, 
        'Assertiveness': assertiveness, 
        'Flexibility': flexibility
    }

In [ ]:
from glob import glob
filepaths = glob('./AIPF Emergenetics Profile/*.pdf')
profiles = []
for fp in filepaths:
    print(fp)
    parsed_pdf = parser.from_file(fp)
    content = parsed_pdf['content']    
    if not content.strip().endswith('Please contact your Emergenetics Associate or email the Emergenetics International\noffice at brains@emergenetics.com with your observations, suggestions, and comments.\n\n \n\nNOTES'):
        print('Invalid file')
        continue
    extracted = extract_values(content)
    profiles.append(extracted)


In [ ]:
import pandas as pd
df_profiles = pd.DataFrame(profiles)
df_profiles

In [ ]:
df_profiles.to_csv('profile.csv', index=False)

In [38]:
import pandas as pd
df_profiles = pd.read_csv('profile.csv')

In [39]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(df_profiles[['Analytical', 'Conceptual', 'Structural', 'Social']])

In [ ]:
import numpy as np
np.save('cosine_similarity.npy', cos_sim)

In [40]:
cos_sim[0][0]

1.0

In [41]:
df_profiles[df_profiles['Name']=='Watson Chua'].index[0]

10

In [58]:
df_cos = pd.DataFrame(cos_sim, index=df_profiles['Name'].values, columns=df_profiles['Name'].values)

In [60]:
df_cos

,Lu Li,Amelia Lee,Quan Xue,Weiguang Chen,Jeffrey Lonan,Pek Kuan Ng,Wenjie Boon,Sulaiman Alwie,Jing Han,Carine Oh,...,Woon Peng Goh,Jiew Peng Lim,Ryan Fong,Esther Gao,Zixiang Ng,Chii Yeh Chin,Alvina Goh,Charlton Lim,Alvin Shi Jiin Ng,Shangru Ng
Lu Li,1.000000,0.955644,0.835436,0.860630,0.663378,0.947208,0.525680,0.687480,0.803047,0.280719,...,0.975111,0.827259,0.812639,0.937289,0.840466,0.927289,0.978877,0.893048,0.883369,0.959152
Amelia Lee,0.955644,1.000000,0.797402,0.965983,0.832722,0.949815,0.642253,0.815728,0.852106,0.546149,...,0.938854,0.925047,0.933429,0.947104,0.852595,0.991770,0.926170,0.964021,0.851304,0.981418
Quan Xue,0.835436,0.797402,1.000000,0.708196,0.534854,0.944414,0.738476,0.593991,0.883206,0.252537,...,0.857594,0.625973,0.807214,0.946831,0.984137,0.832095,0.857325,0.820461,0.982374,0.826387
Weiguang Chen,0.860630,0.965983,0.708196,1.000000,0.947560,0.878219,0.740113,0.924897,0.876005,0.725968,...,0.819218,0.981367,0.953989,0.880815,0.801517,0.969017,0.798158,0.918970,0.785984,0.906749
Jeffrey Lonan,0.663378,0.832722,0.534854,0.947560,1.000000,0.708459,0.795494,0.974660,0.826981,0.877061,...,0.594163,0.959047,0.889045,0.717351,0.665773,0.849447,0.565100,0.775178,0.635330,0.728165
Pek Kuan Ng,0.947208,0.949815,0.944414,0.878219,0.708459,1.000000,0.704278,0.723472,0.898071,0.412413,...,0.957722,0.807613,0.916823,0.999493,0.963752,0.963026,0.950363,0.949861,0.958614,0.962142
Wenjie Boon,0.525680,0.642253,0.738476,0.740113,0.795494,0.704278,1.000000,0.854796,0.926623,0.693305,...,0.474970,0.724713,0.806853,0.721512,0.818210,0.710085,0.454782,0.631260,0.784246,0.558072
Sulaiman Alwie,0.687480,0.815728,0.593991,0.924897,0.974660,0.723472,0.854796,1.000000,0.891532,0.786388,...,0.586253,0.958847,0.851822,0.730670,0.704889,0.828318,0.567152,0.721614,0.708345,0.696648
Jing Han,0.803047,0.852106,0.883206,0.876005,0.826981,0.898071,0.926623,0.891532,1.000000,0.571625,...,0.748334,0.857561,0.895311,0.904948,0.934623,0.884377,0.738939,0.806496,0.939799,0.791877
Carine Oh,0.280719,0.546149,0.252537,0.725968,0.877061,0.412413,0.693305,0.786388,0.571625,1.000000,...,0.263735,0.711200,0.738549,0.432371,0.419883,0.600394,0.215342,0.574789,0.312931,0.449206


In [51]:
df_cos.to_csv('cosine_similarity.csv')

In [59]:
df_cos['Becca Qiu']['Charlton Lim']

0.943907007435672

In [53]:
df_cos.loc['Watson Chua'].nsmallest(5).round(3)

Name
Carine Oh         0.306
Wenjie Boon       0.523
Jeffrey Lonan     0.661
Sulaiman Alwie    0.668
Jing Han          0.794
Name: Watson Chua, dtype: float64

In [11]:
df_profiles['Name']

0              Becca Qiu
1              Carine Oh
2      Alvin Shi Jiin Ng
3             Shangru Ng
4               Quan Xue
5             Zixiang Ng
6             Victor Loh
7            Watson Chua
8             Amelia Lee
9          Woon Peng Goh
10              Jing Han
11        Sulaiman Alwie
12           Pek Kuan Ng
13           Wenjie Boon
14         Jiew Peng Lim
15             Ryan Fong
16                 Lu Li
17            Esther Gao
18         Chii Yeh Chin
19            Alvina Goh
20         Weiguang Chen
Name: Name, dtype: object

In [25]:
import numpy as np
# np.argsort(cos_sim[7])
(-cos_sim[7]).argsort()[1:5]

array([ 0, 16,  9, 19])

In [28]:
df.iloc[19]

Name                      Alvina Goh
Analytical                        48
Conceptual                        45
Structural                         4
Social                             3
Analytical Percentile             95
Conceptual Percentile             89
Structural Percentile              8
Social Percentile                  5
Expressiveness                     7
Assertiveness                     76
Flexibility                        5
Name: 19, dtype: object

In [68]:
df_cos_sim = pd.read_csv('cosine_similarity.csv', index_col=0, header=0)

In [70]:
df_cos_sim

,Lu Li,Amelia Lee,Quan Xue,Weiguang Chen,Jeffrey Lonan,Pek Kuan Ng,Wenjie Boon,Sulaiman Alwie,Jing Han,Hung Siang Ng,...,Woon Peng Goh,Jiew Peng Lim,Ryan Fong,Esther Gao,Zixiang Ng,Chii Yeh Chin,Alvina Goh,Charlton Lim,Alvin Shi Jiin Ng,Shangru Ng
Lu Li,1.000000,0.955644,0.835436,0.860630,0.663378,0.947208,0.525680,0.687480,0.803047,0.690742,...,0.975111,0.827259,0.812639,0.937289,0.840466,0.927289,0.978877,0.893048,0.883369,0.959152
Amelia Lee,0.955644,1.000000,0.797402,0.965983,0.832722,0.949815,0.642253,0.815728,0.852106,0.816938,...,0.938854,0.925047,0.933429,0.947104,0.852595,0.991770,0.926170,0.964021,0.851304,0.981418
Quan Xue,0.835436,0.797402,1.000000,0.708196,0.534854,0.944414,0.738476,0.593991,0.883206,0.648426,...,0.857594,0.625973,0.807214,0.946831,0.984137,0.832095,0.857325,0.820461,0.982374,0.826387
Weiguang Chen,0.860630,0.965983,0.708196,1.000000,0.947560,0.878219,0.740113,0.924897,0.876005,0.920866,...,0.819218,0.981367,0.953989,0.880815,0.801517,0.969017,0.798158,0.918970,0.785984,0.906749
Jeffrey Lonan,0.663378,0.832722,0.534854,0.947560,1.000000,0.708459,0.795494,0.974660,0.826981,0.964901,...,0.594163,0.959047,0.889045,0.717351,0.665773,0.849447,0.565100,0.775178,0.635330,0.728165
Pek Kuan Ng,0.947208,0.949815,0.944414,0.878219,0.708459,1.000000,0.704278,0.723472,0.898071,0.752465,...,0.957722,0.807613,0.916823,0.999493,0.963752,0.963026,0.950363,0.949861,0.958614,0.962142
Wenjie Boon,0.525680,0.642253,0.738476,0.740113,0.795494,0.704278,1.000000,0.854796,0.926623,0.898526,...,0.474970,0.724713,0.806853,0.721512,0.818210,0.710085,0.454782,0.631260,0.784246,0.558072
Sulaiman Alwie,0.687480,0.815728,0.593991,0.924897,0.974660,0.723472,0.854796,1.000000,0.891532,0.995447,...,0.586253,0.958847,0.851822,0.730670,0.704889,0.828318,0.567152,0.721614,0.708345,0.696648
Jing Han,0.803047,0.852106,0.883206,0.876005,0.826981,0.898071,0.926623,0.891532,1.000000,0.923770,...,0.748334,0.857561,0.895311,0.904948,0.934623,0.884377,0.738939,0.806496,0.939799,0.791877
Hung Siang Ng,0.690742,0.816938,0.648426,0.920866,0.964901,0.752465,0.898526,0.995447,0.923770,1.000000,...,0.599605,0.943297,0.873377,0.761301,0.754404,0.839244,0.580010,0.738002,0.750811,0.705233


In [69]:
df_cos_sim.columns

Index(['Lu Li', 'Amelia Lee', 'Quan Xue', 'Weiguang Chen', 'Jeffrey Lonan',
       'Pek Kuan Ng', 'Wenjie Boon', 'Sulaiman Alwie', 'Jing Han',
       'Hung Siang Ng', 'Carine Oh', 'Watson Chua', 'Becca Qiu', 'Victor Loh',
       'Woon Peng Goh', 'Jiew Peng Lim', 'Ryan Fong', 'Esther Gao',
       'Zixiang Ng', 'Chii Yeh Chin', 'Alvina Goh', 'Charlton Lim',
       'Alvin Shi Jiin Ng', 'Shangru Ng'],
      dtype='object')

In [72]:
df_cos_sim['Lu Li']['Lu Li']

1.0